In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipytone

In [ ]:
sr = 44100     # audio sample rate (Hertz)
duration = 4   # audio sample duration (seconds)

time = np.linspace(0, duration, sr * duration)

Compute the wavelengths of the hydrogen atom spectral series using the [Rydberg formula](https://en.wikipedia.org/wiki/Rydberg_formula).

$$ {1 \over \lambda} = R_H \left({1\over n_1}-{1\over n_2}\right) $$

where $ n_2 > n_1 $ and where $ n_1 \in [1, 2, 3...]$ ($n_1 = 1$: Lyman series, $n_1 = 2$: Balmer series)

In [ ]:
def rydberg(time, r_h, n1=1, nseries=8):
    n2 = np.arange(n1 + 1, n1 + 1 + nseries)[:, None]
    series = np.sin((1 / n1**2 - 1 / n2**2) * 2 * np.pi * time * r_h)
    data = np.sum(series, axis=0)
    
    return 0.8 * (data - data.min()) / (data.max() - data.min()) - 0.4
    
    

In [ ]:
plt.plot(rydberg(time, 440));

In [ ]:
def lyman_balmer(time, r_h):
    return rydberg(time, r_h, n1=1) + rydberg(time, r_h, n1=2)

buf = ipytone.AudioBuffer(lyman_balmer(time, 440))

In [ ]:
hydrogen_player = ipytone.Player(buf, volume=-5).to_destination()

In [ ]:
hydrogen_player.start()

In [ ]:
bufs = {
    "C4": ipytone.AudioBuffer(lyman_balmer(time, 261.6)),
    "A4": ipytone.AudioBuffer(lyman_balmer(time, 440)),
    "C5": ipytone.AudioBuffer(lyman_balmer(time, 523.25)),
    "A5": ipytone.AudioBuffer(lyman_balmer(time, 880)),
}

In [ ]:
hydrogen_sampler = ipytone.Sampler(bufs, attack=0.2, release=1.5, volume=-10)
hydrogen_delay = ipytone.PingPongDelay(wet=0)
hydrogen_sampler.chain(hydrogen_delay, ipytone.get_destination())

In [ ]:
hydrogen_sampler.trigger_attack_release(["C4"], 1.0)

In [ ]:
def hydrogen_callback(time, value):
    hydrogen_sampler.trigger_attack_release(value, "2n", time=time)

hydrogen_seq = ipytone.Sequence(
    callback=hydrogen_callback,
    events=[["A3", "A3"], ["A4", None], ["C4", "C4"], ["D4", "F4"]],
    subdivision="1m",
)


In [ ]:
hydrogen_seq.start()

In [ ]:
hydrogen_delay.wet.value = 0.5

In [ ]:
earthquake = np.load("assets/ev0_6.a01.gse2.npy")

In [ ]:
plt.plot(earthquake);

In [ ]:
earthquake_buf = ipytone.AudioBuffer(earthquake)

In [ ]:
earthquake_player = ipytone.Player(earthquake_buf).to_destination()

In [ ]:
earthquake_player.playback_rate = 1.5
earthquake_player.start()

In [ ]:
earthquake_drums = ipytone.Sampler(
    {"A5": earthquake_buf},
    attack=0.02,
    release=0.8,
    volume=-3,
)

earthquake_eq = ipytone.EQ3(low=5, mid=-4, high=4, low_frequency=150)
earthquake_comp = ipytone.Compressor(attack=0.3, ratio=5)
earthquake_dist = ipytone.Distortion(wet=0.05, oversample="none")
earthquake_delay = ipytone.PingPongDelay(wet=0.1)
earthquake_reverb = ipytone.Reverb(wet=0.2, decay=8, pre_delay=0.5)

earthquake_drums.chain(
    earthquake_eq,
    earthquake_comp,
    earthquake_dist,
    earthquake_delay,
    earthquake_reverb,
    ipytone.destination
)

In [ ]:
earthquake_drums.trigger_attack_release("E3", "8n")

In [ ]:
earthquake_drums.trigger_attack_release("G5", "8n")

In [ ]:
earthquake_dist.wet.value = 0.05
earthquake_dist.oversample = "none"

earthquake_eq.low_frequency.value = 150

earthquake_eq.mid.value = -4

earthquake_reverb.wet.value = 0.1
earthquake_reverb.decay = 8
earthquake_reverb.pre_delay = 0.5

In [ ]:
import ipycanvas

height = 300

canvas = ipycanvas.MultiCanvas(n_canvases=3, width=600, height=height)

canvas[0].fill_style = "blue"
canvas[0].fill_rect(0, 0, 200, height=height)
canvas[0].fill_style = "red"
canvas[0].fill_rect(400, 0, 200, height=height)

canvas[1].global_alpha = 0.7
canvas[1].fill_style = "white"
canvas[2].fill_style = "black"

canvas

In [ ]:
import random

last_played = ""

def play_earthquake(x, y):
    global last_played, height

    canvas[1].clear()
    canvas[2].clear()
    
    velocity = (height - y) / height
    canvas[2].stroke_circle(
        600 // 2,
        height // 2,
        max(1, velocity * 100)
    )

    # kick
    if x < 200:
        canvas[1].fill_rect(0, 0, 200, height=height)
        if last_played != "kick":
            last_played = "kick"
            earthquake_drums.trigger_attack_release(
                "E3", "16n", time="@16n"
            )
    
    # snare
    elif x > 400:
        canvas[1].fill_rect(400, 0, 200, height=height)
        if last_played != "snare":
            last_played = "snare"
            earthquake_drums.trigger_attack_release(
                "B5", "16n", time="@16n"
            )
    
    # hi-hats
    elif x > 200 and x <= 400:
        last_played = "hh"
        earthquake_drums.trigger_attack_release(
            1e4 + random.random() * 1e3,
            0.1,
            time="@16n",
            velocity=velocity,
        )


In [ ]:
canvas[2].on_mouse_move(play_earthquake)

In [ ]:
hydrogen_sampler2 = ipytone.Sampler(bufs, attack=0.003, release=0.2, volume=0)
hydrogen_dist2 = ipytone.Distortion(wet=0.2, distortion=0.1)
hydrogen_sampler2.chain(hydrogen_dist2, ipytone.get_destination())

In [ ]:
hydrogen_sampler2.trigger_attack_release("A2", 0.1)

In [ ]:
def hydrogen_callback2(time, value):
    hydrogen_sampler2.trigger_attack_release(value, 0.15, time=time)

hydrogen_seq2 = ipytone.Sequence(
    callback=hydrogen_callback2,
    events=[
        ["A2", "A2", "A2", "A2"],
        ["A2", "C3", "A2", "A2"],
        ["A3", "A2", "A2", "A2"],
        ["A2", "C3", "A2", "A2"],
        ["C2", "C2", "C2", "C2"],
        ["C2", "C2", "C2", "C2"],
        ["D2", "D2", "D2", "D2"],
        ["F2", "F2", "F2", "F2"],
    ],
    subdivision="2n",
)

In [ ]:
hydrogen_seq2.start()

In [ ]:
hydrogen_seq2.dispose()

In [ ]:
hydrogen_dist2.wet.value = 0.2
hydrogen_dist2.distortion = 0.1
hydrogen_dist2.oversample = "none"

In [ ]:
hydrogen_sampler2.volume.value = 2

In [ ]:
bufs2 = {
    "A4": ipytone.AudioBuffer(rydberg(time, 440, n1=1, nseries=4)),
}

In [ ]:
hydrogen_sampler3 = ipytone.Sampler(
    urls={"A4": ipytone.AudioBuffer(rydberg(time, 440, n1=1, nseries=4))},
    attack=0.1,
    release=0.05,
    volume=-5,
)
hydrogen_tremolo3 = ipytone.Tremolo()
hydrogen_sampler3.chain(hydrogen_tremolo3, ipytone.get_destination())

In [ ]:
def hydrogen_callback3(time, value):
    hydrogen_sampler3.trigger_attack_release(value, 0.03, time=time)

hydrogen_seq3 = ipytone.Sequence(
    callback=hydrogen_callback3,
    events=[
        [None, None, None, "A5"],
        [None, "A6", "A5", None],
        [None, None, "A7", None],
        [None, "A6", "A5", None],
    ],
    subdivision="4n",
)

In [ ]:
hydrogen_seq3.start()

In [ ]:
hydrogen_sampler3.trigger_attack_release("A7", 0.03)